Dans ce script, nous allons tester différents modèles de type convolution Nelle pour examiner leur performance. Nous allons ensuite tester s'ils sont robustes lorsqu'on applique une rotation à l'image d'entrée.

# testing each network for different rotations

In [1]:
import retinoto_py as fovea
args = fovea.Params(do_mask=True)
args

Welcome on macOS-26.1-arm64-arm-64bit-Mach-O	 user laurent	Running on MPS device (Apple Silicon/MacOS)	 - macos_version = 26.1	 with device mps, pytorch==2.9.1
Random seed 1998 has been set.


Params(image_size=224, do_mask=True, do_fovea=False, rs_min=0.0, rs_max=-5.0, padding_mode='zeros', seed=1998, batch_size=64, num_workers=4, in_memory=True, model_name='resnet101', do_scratch=False, num_epochs=10, n_train_stop=32768, n_val_stop=4096, lr=0.003, delta1=0.03, delta2=0.0, weight_decay=0.01, label_smoothing=0.0001, shuffle=True, verbose=True)

In [2]:
all_angles = fovea.np.linspace(0, 180, 25)
all_angles

array([  0. ,   7.5,  15. ,  22.5,  30. ,  37.5,  45. ,  52.5,  60. ,
        67.5,  75. ,  82.5,  90. ,  97.5, 105. , 112.5, 120. , 127.5,
       135. , 142.5, 150. , 157.5, 165. , 172.5, 180. ])

In [3]:
delta_angle = all_angles[1] - all_angles[0]
all_angles_min = all_angles - delta_angle / 2
all_angles_min, all_angles_min + delta_angle, delta_angle

(array([ -3.75,   3.75,  11.25,  18.75,  26.25,  33.75,  41.25,  48.75,
         56.25,  63.75,  71.25,  78.75,  86.25,  93.75, 101.25, 108.75,
        116.25, 123.75, 131.25, 138.75, 146.25, 153.75, 161.25, 168.75,
        176.25]),
 array([  3.75,  11.25,  18.75,  26.25,  33.75,  41.25,  48.75,  56.25,
         63.75,  71.25,  78.75,  86.25,  93.75, 101.25, 108.75, 116.25,
        123.75, 131.25, 138.75, 146.25, 153.75, 161.25, 168.75, 176.25,
        183.75]),
 np.float64(7.5))

In [4]:
for model_name in  fovea.all_model_names:
    print(f'{model_name=}')

model_name='resnet18'
model_name='resnet50'
model_name='resnet101'


In [5]:
for angles_min, angles_max in zip(all_angles_min, all_angles_min + delta_angle):
    print(f'{fovea.np.mod(angles_min,360)=:.3f}\t{fovea.np.mod(angles_max, 360)=:.3f}')

fovea.np.mod(angles_min,360)=356.250	fovea.np.mod(angles_max, 360)=3.750
fovea.np.mod(angles_min,360)=3.750	fovea.np.mod(angles_max, 360)=11.250
fovea.np.mod(angles_min,360)=11.250	fovea.np.mod(angles_max, 360)=18.750
fovea.np.mod(angles_min,360)=18.750	fovea.np.mod(angles_max, 360)=26.250
fovea.np.mod(angles_min,360)=26.250	fovea.np.mod(angles_max, 360)=33.750
fovea.np.mod(angles_min,360)=33.750	fovea.np.mod(angles_max, 360)=41.250
fovea.np.mod(angles_min,360)=41.250	fovea.np.mod(angles_max, 360)=48.750
fovea.np.mod(angles_min,360)=48.750	fovea.np.mod(angles_max, 360)=56.250
fovea.np.mod(angles_min,360)=56.250	fovea.np.mod(angles_max, 360)=63.750
fovea.np.mod(angles_min,360)=63.750	fovea.np.mod(angles_max, 360)=71.250
fovea.np.mod(angles_min,360)=71.250	fovea.np.mod(angles_max, 360)=78.750
fovea.np.mod(angles_min,360)=78.750	fovea.np.mod(angles_max, 360)=86.250
fovea.np.mod(angles_min,360)=86.250	fovea.np.mod(angles_max, 360)=93.750
fovea.np.mod(angles_min,360)=93.750	fovea.np.mod(ang

In [ ]:
for dataset in fovea.all_datasets:
    for model_name in fovea.all_model_names:
        npz_filename = args.data_cache / f'17_rotation-attack_name={model_name}_dataset={dataset}.npz'
        print(f'Rotation attack for {model_name=} and {dataset=} \t {npz_filename=}')

        # %rm {npz_filename}  # FORCING RECOMPUTE

        if npz_filename.exists():
            with fovea.np.load(npz_filename) as data:
                # --- 3. Access the arrays inside the 'with' block ---
                # It's good practice to convert to tensors (or whatever final format you need)
                # right away, so you don't need to keep the file open.
                attack_angle = data['attack_angle']
                attack_success = data['attack_success']
                all_results = data['results']
        else:
            VAL_DATA_DIR = args.DATAROOT / f'Imagenet_{dataset}' / 'val'
            args = fovea.Params(shuffle=False, do_mask=True, model_name=model_name, batch_size=500)
            model = fovea.load_model(args)
            model.eval()

            n_val_stop = args.n_val_stop
            if n_val_stop==0: n_val_stop = len(val_loader.dataset)

            # https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html 
            criterion = fovea.nn.CrossEntropyLoss(reduction='none')
            # dataset_len = len(val_loader.dataset)
            all_results = fovea.np.empty((len(all_angles), n_val_stop)) 
            all_logits = fovea.np.empty((len(all_angles), n_val_stop, model.fc.out_features)) 
            all_losses = fovea.np.empty((len(all_angles), n_val_stop)) 
            all_labels = []
            outer_progress = fovea.tqdm(
                                    enumerate(zip(all_angles_min, all_angles_min + delta_angle)),
                                    total=len(all_angles_min),
                                    desc="Validating Angles", 
                                    leave=False
                                )
            for i_angle, (angle_min, angle_max) in outer_progress:
                val_dataset, class_to_idx, idx_to_class = fovea.get_dataset(args, VAL_DATA_DIR, 
                                                                    angle_min=angle_min, angle_max=angle_max, in_memory=True, n_stop=n_val_stop)
                val_loader = fovea.get_loader(args, val_dataset)

                correct_predictions = 0
                total_predictions = 0

                image_count = 0
                for images, true_labels in fovea.tqdm(val_loader, desc=f"Angle [{.5*(angle_min+angle_max):.1f}°]", position=1, leave=False):
                    images = images.to(args.device)
                    true_labels = true_labels.to(args.device)

                    # Get predictions (no need for gradients)
                    with fovea.torch.no_grad():
                        if i_angle==0: all_labels.extend([l.item() for l in true_labels])
                        outputs = model(images)
                        all_logits[i_angle, image_count:(image_count + images.size(0)), :] = outputs.cpu().numpy()
                        _, predicted_labels = fovea.torch.max(outputs, dim=1)
                        correct_predictions_in_batch = (predicted_labels == true_labels)
                        all_results[i_angle, image_count:image_count + images.size(0)] = correct_predictions_in_batch.cpu().numpy()
                        loss = criterion(outputs, true_labels)
                        all_losses[i_angle, image_count:(image_count + images.size(0))] = loss.cpu().numpy()
                    image_count += images.size(0)
                outer_progress.set_postfix(last_acc=f"{all_results[i_angle, :].mean():.4f}")

            attack_angle = all_losses.argmax(axis=0)
            attack_idx = all_logits[attack_angle[:len(all_labels)], fovea.np.arange(len(all_labels)), :].argmax(axis=1)
            attack_success = (attack_idx == all_labels)

            fovea.np.savez(npz_filename, 
                    attack_angle=attack_angle,
                    attack_success=attack_success,
                    results=all_results)



Rotation attack for model_name='resnet18' and dataset='full' 	 npz_filename=PosixPath('cached_data/17_rotation-attack_name=resnet18_dataset=full.npz')
Rotation attack for model_name='resnet50' and dataset='full' 	 npz_filename=PosixPath('cached_data/17_rotation-attack_name=resnet50_dataset=full.npz')
Rotation attack for model_name='resnet101' and dataset='full' 	 npz_filename=PosixPath('cached_data/17_rotation-attack_name=resnet101_dataset=full.npz')
Rotation attack for model_name='resnet18' and dataset='bbox' 	 npz_filename=PosixPath('cached_data/17_rotation-attack_name=resnet18_dataset=bbox.npz')
Rotation attack for model_name='resnet50' and dataset='bbox' 	 npz_filename=PosixPath('cached_data/17_rotation-attack_name=resnet50_dataset=bbox.npz')


Validating Angles:   0%|          | 0/25 [00:00<?, ?it/s]

Putting images in memory:   0%|          | 0/4096 [00:00<?, ?it/s]

Angle [0.0°]:   0%|          | 0/8 [00:00<?, ?it/s]

# analysis: average accuracy for different rotations

In [ ]:
all_results.shape

In [ ]:
all_results.mean(axis=1)

In [ ]:

# --- Création du Plot ---
fig, ax = fovea.plt.subplots()

for dataset, color in zip(fovea.all_datasets, fovea.all_datasets_color):
    for model_name, ls in zip(fovea.all_model_names, fovea.all_model_names_ls):
        npz_filename = args.data_cache / f'17_rotation-attack_{model_name=}_{dataset=}.npz'
        with fovea.np.load(npz_filename) as data:
            all_results = data['results']
        # `all_results` est un tableau de booléens (True/False pour correct/incorrect)
        # Pour calculer la précision, on calcule la moyenne (True=1, False=0) sur l'axe des images (axe=1)
        mean_accuracy_per_angle = all_results.mean(axis=1)

        # Tracer l'accuracy moyenne en fonction de l'angle de référence
        ax.plot(all_angles, mean_accuracy_per_angle, marker='o', linestyle=ls, color=color, lw=2, markersize=8, label=model_name if dataset=='full' else '')

# --- Finalisation du Graphique ---
ax.set_xlabel("Angle de rotation de l'image (degrés)")
ax.set_ylabel("Accuracy moyenne")
ax.set_title(f"Robustesse du modèle ({args.model_name}) aux rotations")
ax.set_ylim(0, 1.05) # S'assurer que l'axe des y est bien entre 0 et 1

# On peut ajouter une ligne horizontale pour la performance de base (angle=0)
# if 0 in all_angles[:-1]:
#     base_accuracy = mean_accuracy_per_angle[all_angles == 0][0]
#     plt.axhline(y=base_accuracy, color='r', linestyle='--', label=f'Performance de base (angle=0): {base_accuracy:.2f}')
#     plt.legend()


In [ ]:
all_angles

In [ ]:
all_angles_wrap = fovea.np.hstack((all_angles, fovea.np.mod(all_angles[1:]+180, 360)))
all_angles_wrap

In [ ]:
mean_accuracy_per_angle_wrap = fovea.np.hstack((mean_accuracy_per_angle, mean_accuracy_per_angle[:-1][::-1]))
mean_accuracy_per_angle_wrap

In [ ]:
# --- Création du Polar Plot ---

# 1. Créer la figure et l'axe en projection polaire
fig = fovea.plt.figure()
ax = fig.add_subplot(111, polar=True)

# 2. Convertir les angles de degrés en radians (nécessaire pour Matplotlib)
angles_rad = fovea.np.deg2rad(all_angles_wrap)

for dataset, color in zip(fovea.all_datasets, fovea.all_datasets_color):
    for model_name, ls in zip(fovea.all_model_names, fovea.all_model_names_ls):
        npz_filename = args.data_cache / f'17_rotation-attack_{model_name=}_{dataset=}.npz'
        with fovea.np.load(npz_filename) as data:
            all_results = data['results']

        mean_accuracy_per_angle = all_results.mean(axis=1)
        ax.plot(angles_rad, fovea.np.sqrt(mean_accuracy_per_angle_wrap), marker='o', linestyle='-', lw=2, markersize=8, color='r')
        ax.fill(angles_rad, fovea.np.sqrt(mean_accuracy_per_angle_wrap), alpha=0.25, color='r')

# --- Finalisation du Graphique Polaire ---
# Définir la direction du zéro (haut) et le sens horaire/anti-horaire
ax.set_theta_zero_location('N')
ax.set_theta_direction(-1) # -1 pour horaire, 1 pour anti-horaire

# Définir les étiquettes des angles sur le cercle
# S'assurer que les étiquettes sont alignées
ax.set_thetagrids(fovea.np.degrees(angles_rad), labels=[f'{a:.0f}°' for a in all_angles_wrap])

# Définir les limites et les étiquettes de l'axe radial (l'accuracy)
# ax.set_xlim(0)
# ax.set_yscale('logit')
ax.set_ylim(0, 1.05)
# ax.set_ylim(0)
# ax.set_ylabel("Accuracy moyenne", labelpad=30) # labelpad pour éloigner l'étiquette du graphique
ax.set_rlabel_position(45) # Positionner l'étiquette de l'axe radial

ax.set_yticks([])

# Ajouter un titre
# ax.set_title(f"Robustesse du modèle ({args.model_name}) aux rotations\n(Accuracy vs. Angle)", pad=20)

# Ajouter la ligne de performance de base (angle=0)
# Il faut trouver l'index de l'angle le plus proche de 0
zero_angle_idx = (fovea.np.abs(all_angles)).argmin()
base_accuracy = mean_accuracy_per_angle[zero_angle_idx]
# ax.plot(angles_rad[zero_angle_idx], base_accuracy, 'rD', markersize=12, label=f'Performance de base: {base_accuracy:.2f}')
ax.plot(angles_rad, base_accuracy*fovea.np.ones_like(mean_accuracy_per_angle_wrap), '--', lw=1.5, alpha=.3, color='red')
ax.legend(loc='lower left', bbox_to_anchor=(1.05, 1.1))

# analysis: average accuracy for different rotations

In [ ]:
attack_angle.shape, attack_success.shape

In [ ]:
fovea.np.unique(attack_angle)

In [ ]:
all_angles

In [ ]:
fovea.sns.displot(x=all_angles, y=attack_angle)

In [ ]:
attack_success

In [ ]:
attack_success.mean()